In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/Users/leduongkhoa/Regression_MachineLearning_End2End/data/raw/untouched_raw_original.csv")

In [3]:
pd.set_option('display.max_columns', None) # show all cols
pd.set_option('display.max_rows', 100)    # show max 100 rows
# pd.options.display.max_rows = 100

In [4]:
df.shape

(884092, 39)

In [5]:
df.head(5)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
1,2012-04-30,61870.0,245000.0,40.723982,130.528256,22.0,29.0,56.0,69.0,89.5,0.946642,0.090909,0.034483,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202421.064584,Atlanta-Sandy Springs-Alpharetta
2,2012-05-31,125500.0,217450.0,63.913043,119.919216,24.0,40.0,63.0,60.0,144.5,0.955624,0.208333,0.100000,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202681.309539,Atlanta-Sandy Springs-Alpharetta
3,2012-06-30,153000.0,189900.0,81.598080,105.617353,34.0,46.0,50.0,57.0,126.0,0.970608,0.176471,0.108696,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202998.603897,Atlanta-Sandy Springs-Alpharetta
4,2012-07-31,165500.0,154000.0,81.598080,83.921175,39.0,49.0,42.0,50.0,80.0,0.982105,0.256410,0.102041,ATL,30002,2012,12.0,2.0,3.0,1.0,60.0,45.0,57.0,4.0,6.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,203781.903446,Atlanta-Sandy Springs-Alpharetta


In [6]:
df['date'] = pd.to_datetime(df['date'])
print(df['date'].dt.year.value_counts().sort_index())

2012    62260
2013    74712
2014    74712
2015    74712
2016    74712
2017    74712
2018    74712
2019    74712
2020    74712
2021    74712
2022    74712
2023    74712
Name: date, dtype: int64


In [7]:
df = df.sort_values(by=['date']).reset_index(drop=True)

### Avoid Leakage

* **Train set (2012–2019):** This is "clean" historical training data.
* **Eval set (2020–2021):** This data is "unseen" during the actual training process, but it is still used for validation and hyperparameter tuning.
* **Holdout set (2022–2023):** This set remains completely untouched until the very end of the project to ensure an unbiased final evaluation.

In [8]:
evel_cut_off_date = pd.to_datetime("2020-01-01") # YYYY-MM-DD format
holdout_cut_off_date = pd.to_datetime("2022-01-01") # YYYY-MM-DD format

In [9]:
train_df = df[df['date'] < evel_cut_off_date]
eval_df = df[(df['date'] >= evel_cut_off_date) & (df['date'] < holdout_cut_off_date)]
holdout_df = df[df['date'] >= holdout_cut_off_date]

print(f"Train_df shape: {train_df.shape}")
print(f"Eval_df shape: {eval_df.shape}")
print(f"Holdout_df shape: {holdout_df.shape}")

Train_df shape: (585244, 39)
Eval_df shape: (149424, 39)
Holdout_df shape: (149424, 39)


In [10]:
train_df.head(3)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
1,2012-03-31,200000.0,7500.0,104.931794,79.265873,1.0,1.0,1.0,2.0,290.0,0.909091,0.000000,0.000000,PGH,15469,2012,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2441.0,41.8,20241.0,2385.0,1108.0,641.0,94600.0,1171.0,52.0,2376.0,2376.0,1018.0,105863.681174,Pittsburgh
2,2012-03-31,75750.0,92400.0,165.898618,63.412698,2.0,2.0,4.0,13.0,193.0,0.986815,0.000000,0.000000,PGH,15468,2012,0.0,0.0,0.0,0.0,1.0,0.0,5.0,0.0,0.0,2171.0,47.8,18740.0,2094.0,1252.0,627.0,62800.0,894.0,30.0,2106.0,2106.0,807.0,60152.096646,Pittsburgh


In [11]:
eval_df.head(3)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
585244,2020-01-31,184000.0,168450.0,123.333333,109.899800,35.0,30.0,34.0,37.0,50.0,0.954732,0.171429,0.300000,CHI,60482,2020,3.0,0.0,2.0,0.0,68.0,30.0,60.0,5.0,14.0,10849.0,42.1,27037.0,10795.0,4598.0,964.0,169000.0,5690.0,414.0,10573.0,10573.0,5089.0,1.648397e+05,Chicago-Naperville-Elgin
585245,2020-01-31,140000.0,139000.0,94.691957,111.311054,61.0,81.0,83.0,54.0,10.0,0.963039,0.098361,0.469136,CIN,41071,2020,35.0,5.0,4.0,3.0,139.0,259.0,86.0,1.0,8.0,20624.0,36.6,30324.0,19423.0,10164.0,802.0,132400.0,10479.0,552.0,19809.0,19809.0,9383.0,1.321643e+05,Cincinnati
585246,2020-01-31,1040000.0,1600000.0,1386.111111,1801.309410,134.0,151.0,210.0,563.0,188.0,0.971567,0.037313,0.006623,NY,10023,2020,344.0,11.0,26.0,8.0,404.0,2054.0,324.0,114.0,172.0,62541.0,43.3,126937.0,60995.0,41353.0,2346.0,1218900.0,37308.0,1440.0,60206.0,60206.0,31554.0,1.494755e+06,New York-Newark-Jersey City


In [12]:
holdout_df.head(3)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
734668,2022-01-31,340000.0,349999.0,189.867865,195.526649,282.0,287.0,241.0,73.0,12.0,0.991177,0.308511,0.634146,ORL,32771,2022,9.0,0.0,1.0,1.0,56.0,64.0,18.0,4.0,9.0,57074.0,37.7,33896.0,56681.0,22246.0,1289.0,261800.0,29485.0,1469.0,55621.0,55621.0,24381.0,314667.746005,Orlando-Kissimmee-Sanford
734669,2022-01-31,310000.0,269900.0,243.839836,239.543726,49.0,47.0,43.0,7.0,6.5,1.006586,0.489796,0.723404,TPA,33778,2022,21.0,0.0,3.0,2.0,35.0,57.0,32.0,0.0,22.0,15912.0,47.8,31277.0,15405.0,7865.0,1161.0,206600.0,7846.0,345.0,15423.0,15423.0,6733.0,325271.243813,Tampa-St. Petersburg-Clearwater
734670,2022-01-31,274950.0,299450.0,175.320145,287.500000,4.0,3.0,6.0,2.0,33.0,0.975694,0.000000,0.666667,PHL,21914,2022,3.0,0.0,0.0,0.0,8.0,15.0,7.0,0.0,2.0,725.0,51.1,36407.0,725.0,477.0,1393.0,265000.0,268.0,12.0,714.0,714.0,250.0,278119.378304,Philadelphia-Camden-Wilmington


In [13]:
train_df.to_csv("/Users/leduongkhoa/Regression_MachineLearning_End2End/data/raw/train.csv", index=False)
eval_df.to_csv("/Users/leduongkhoa/Regression_MachineLearning_End2End/data/raw/eval.csv", index=False)
holdout_df.to_csv("/Users/leduongkhoa/Regression_MachineLearning_End2End/data/raw/holdout.csv", index=False)